In [22]:
import minsearch
import json
from tqdm.auto import tqdm
import tiktoken
import openai
from openai import OpenAI
import os

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch('http://localhost:9200') 

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-2"

In [ ]:
es_client.indices.create(index=index_name, body=index_settings)

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [8]:
query = "How do I execute a command in a running docker container?"

search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [9]:
response['hits']['hits'][0]['_score']

75.66571

## Q4. Filtering

In [10]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [11]:
for source in response['hits']['hits'][0:3]:
    print(source['_source']['question'])

How do I debug a docker container?
How do I debug a docker container?
How do I copy files from my local machine to docker container?


## Q5. Building a prompt

In [12]:
response = es_client.search(index=index_name, body=search_query)
    
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from

In [13]:
question = "How do I execute a command in a running docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = ""
    
for doc in result_docs:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

len(prompt_template.format(question=query, context=context).strip())

2549

## Q6. Building a prompt

In [14]:
encoding = tiktoken.encoding_for_model("gpt-4o")

Use the encode function. How many tokens does our prompt have?

In [19]:
len(encoding.encode(prompt_template.format(question=query, context=context).strip()))

556

## Q7. generating the answer

In [28]:
client = OpenAI()
prompt = prompt_template.format(question=query, context=context).strip()
response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

In [31]:
print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. Here's a step-by-step process:

1. First, list all running containers to find the container ID:
   ```sh
   docker ps
   ```

2. Once you have identified the container ID for the specific container, execute the desired command inside the container using:
   ```sh
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session inside the container, you would use:
   ```sh
   docker exec -it <container-id> bash
   ```
